# Lecture 3

In [1]:
model_path = "C:/Users/user/Downloads/hand_landmarker.task"

In [2]:
import mediapipe as mp
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import time

In [3]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the image mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
    min_hand_detection_confidence=0.3,
    num_hands=2)

In [4]:
def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

  return annotated_image

In [5]:
def HandDirection(landmarks):
    # To Determine if the hand is pointing up or left or right
    wrist = landmarks[1]
    IndexFinger = landmarks[5]
    MiddleFinger = landmarks[9]
    RingFinger = landmarks[13]
    PinkyFinger = landmarks[17]
    Fingers = [IndexFinger, MiddleFinger, RingFinger, PinkyFinger]

    fingers_counter = 0
    # up
    for Coordinates in Fingers:
        if Coordinates.y < wrist.y:
            fingers_counter += 1
    if fingers_counter == 4:
        return "Up"


    fingers_counter = 0
    # Right
    for Coordinates in Fingers:
        if Coordinates.x > wrist.x:
            fingers_counter += 1
    if fingers_counter == 4:
        return "Right"

    fingers_counter = 0
    # Left
    for Coordinates in Fingers:
        if Coordinates.x < wrist.x:
            fingers_counter += 1
    if fingers_counter == 4:
        return "Left"


In [6]:
def MoveType(landmarks, Direction):
    # Index Finger
    Hand_5 = landmarks[5]
    Hand_6 = landmarks[6]
    Hand_8 = landmarks[8]
    # Middle Finger
    Hand_9 = landmarks[9]
    Hand_10 = landmarks[10]
    Hand_12 = landmarks[12]
    # Ring Finger
    Hand_13 = landmarks[13]
    Hand_14 = landmarks[14]
    Hand_16 = landmarks[16]
    # Pinky Finger
    Hand_17 = landmarks[17]
    Hand_18 = landmarks[18]
    Hand_20 = landmarks[20]


    Move = "Unknown"
    # if Direction is up
    if Direction == "Up":
        if Hand_5.y > Hand_8.y and Hand_9.y > Hand_12.y and Hand_13.y > Hand_16.y and Hand_17.y > Hand_20.y:
            return "Paper"
        elif Hand_5.y > Hand_8.y and Hand_9.y > Hand_12.y and Hand_14.y < Hand_16.y and Hand_18.y < Hand_20.y:
            return "Scissors"
        elif Hand_6.y < Hand_8.y and Hand_10.y < Hand_12.y and Hand_14.y < Hand_16.y and Hand_18.y < Hand_20.y:
            return "Rock"
        

    # if Direction is right
    elif Direction == "Right":
        if Hand_5.x < Hand_8.x and Hand_9.x < Hand_12.x and Hand_13.x < Hand_16.x and Hand_17.x < Hand_20.x:
            return "Paper"
        elif Hand_5.x < Hand_8.x and Hand_9.x < Hand_12.x and Hand_14.x > Hand_16.x and Hand_18.x > Hand_20.x:
            return "Scissors"
        elif Hand_6.x > Hand_8.x and Hand_10.x > Hand_12.x and Hand_14.x > Hand_16.x and Hand_18.x > Hand_20.x:
            return "Rock"

    # if Direction is left
    elif Direction == "Left":
        if Hand_5.x > Hand_8.x and Hand_9.x > Hand_12.x and Hand_13.x > Hand_16.x and Hand_17.x > Hand_20.x:
            return "Paper"
        elif Hand_6.x > Hand_10.x and Hand_9.x > Hand_12.x and Hand_14.x < Hand_16.x and Hand_18.x < Hand_20.x:
            return "Scissors"
        elif Hand_6.x < Hand_8.x and Hand_10.x < Hand_12.x and Hand_14.x < Hand_16.x and Hand_18.x < Hand_20.x:
            return "Rock"
    return Move

In [7]:
def ComputerMove():
    import random
    moves = ["Rock", "Paper", "Scissors"]
    return random.choice(moves)

In [8]:
def Winner(PlayerMove, ComputerMove):
    if PlayerMove == ComputerMove:
        return "Draw"
    elif PlayerMove == "Rock" and ComputerMove == "Scissors":
        return "Player"
    elif PlayerMove == "Paper" and ComputerMove == "Rock":
        return "Player"
    elif PlayerMove == "Scissors" and ComputerMove == "Paper":
        return "Player"
    else:
        return "Computer"

In [9]:
def CenterTheText(frame ,text, font, scale, thickness):
    textsize = cv2.getTextSize(text, font, scale, thickness)[0]
    textX = (frame.shape[1] - textsize[0]) // 2
    textY = (frame.shape[0] + textsize[1]) // 2
    return textX, textY


In [10]:
def AddMoveImage(frame, Move):
    if Move == "Rock":
        Rock = cv2.imread("Images/Rock.png")
        Rock = cv2.resize(Rock, (200, 200))
        y = frame.shape[0] - 200
        x = (frame.shape[1] - 200) // 2
        frame[y:y+200, x:x+200] = Rock
    elif Move == "Paper":
        Paper = cv2.imread("Images/Paper.png")
        Paper = cv2.resize(Paper, (200, 200))
        y = frame.shape[0] - 200
        x = (frame.shape[1] - 200) // 2
        frame[y:y+200, x:x+200] = Paper
    elif Move == "Scissors":
        Scissors = cv2.imread("Images/Scissor.png")
        Scissors = cv2.resize(Scissors, (200, 200))
        y = frame.shape[0] - 200
        x = (frame.shape[1] - 200) // 2
        frame[y:y+200, x:x+200] = Scissors
    return frame

In [ ]:
video = cv2.VideoCapture(0)

while True:
    success, frame = video.read()
    if not success:
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    with HandLandmarker.create_from_options(options) as landmarker:
        hand_landmarker_result = landmarker.detect(mp_image)
        
        
        if hand_landmarker_result:
            if hand_landmarker_result.hand_landmarks:
                annotated_image = draw_landmarks_on_image(frame, hand_landmarker_result)
                Direction = HandDirection(hand_landmarker_result.hand_landmarks[0])
                Move = MoveType(hand_landmarker_result.hand_landmarks[0], Direction)
                cv2.putText(annotated_image, Direction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(annotated_image, Move, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                Computer = ComputerMove()
                cv2.putText(annotated_image, Computer, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                Winner_ = Winner(Move, Computer)
                cv2.putText(annotated_image, Winner_, (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.imshow("Frame", annotated_image)
            else:
                cv2.imshow("Frame", frame)
                
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):   
        break

cv2.destroyAllWindows()
video.release()

In [11]:
video = cv2.VideoCapture(0)

Play = True
while Play:
    success, frame = video.read()
    if not success:
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    Start = False
    frame_copy = frame.copy()
    # Make the middle of the text to be the middle of the frame
    X, Y = CenterTheText(frame_copy, "Press 's' to start the game", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.putText(frame_copy, "Press 's' to start the game", (X, Y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Frame", frame_copy)
    if cv2.waitKey(1) & 0xFF == ord('s'):
        Start = True
    if Start:
            # Start a timer to determine the time taken to make a move
            Timer = time.time()
            while time.time() - Timer < 3:

                success, frame = video.read()
                frame_copy = frame.copy()
                
                if time.time() - Timer < 1:
                    cv2.putText(frame, "3", (int(frame.shape[1]/2), int(frame.shape[0]/2)), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
                elif time.time() - Timer < 2:
                    cv2.putText(frame, "2", (int(frame.shape[1]/2), int(frame.shape[0]/2)), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
                elif time.time() - Timer < 3:
                    cv2.putText(frame, "1", (int(frame.shape[1]/2), int(frame.shape[0]/2)), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
                
                mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                with HandLandmarker.create_from_options(options) as landmarker:
                    hand_landmarker_result = landmarker.detect(mp_image)

                if hand_landmarker_result:
                    if hand_landmarker_result.hand_landmarks:

                        annotated_image = draw_landmarks_on_image(frame, hand_landmarker_result)
                        Direction = HandDirection(hand_landmarker_result.hand_landmarks[0])
                        Move = MoveType(hand_landmarker_result.hand_landmarks[0], Direction)

                        cv2.putText(annotated_image, Direction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        cv2.putText(annotated_image, Move, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                        cv2.imshow("Frame", annotated_image)
                    else:
                        cv2.imshow("Frame", frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            # After the time is up, the player makes a move
            if hand_landmarker_result:
                    if hand_landmarker_result.hand_landmarks:

                        Direction = HandDirection(hand_landmarker_result.hand_landmarks[0])
                        Move = MoveType(hand_landmarker_result.hand_landmarks[0], Direction)
                        Computer = ComputerMove()
                        Winner_ = Winner(Move, Computer)

                        while True:
                            success, frame = video.read()

                            if Winner_ == "Player":
                                X, Y = CenterTheText(frame, "You Win !!", cv2.FONT_HERSHEY_SIMPLEX, 3, 5)
                                cv2.putText(frame, "You Win !!", (X, Y-100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
                                X, Y = CenterTheText(frame, f"Computer Move was: {Computer}", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
                                cv2.putText(frame, f"Computer Move was: {Computer}", (X, Y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                                frame = AddMoveImage(frame, Computer)
                            elif Winner_ == "Computer":
                                X, Y = CenterTheText(frame, "You Lost..", cv2.FONT_HERSHEY_SIMPLEX, 3, 5)
                                cv2.putText(frame, "You Lost..", (X, Y-100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
                                X, Y = CenterTheText(frame, f"Computer Move was: {Computer}", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
                                cv2.putText(frame, f"Computer Move was: {Computer}", (X, Y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                                frame = AddMoveImage(frame, Computer)
                            elif Winner_ == "Draw":
                                X, Y = CenterTheText(frame, "Draw", cv2.FONT_HERSHEY_SIMPLEX, 3, 5)
                                cv2.putText(frame, "Draw", (X, Y-100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
                                X, Y = CenterTheText(frame, f"Computer Move was: {Computer}", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
                                cv2.putText(frame, f"Computer Move was: {Computer}", (X, Y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                                frame = AddMoveImage(frame, Computer)

                            X, Y = CenterTheText(frame, "Do you want to play again? (y/n)", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
                            cv2.putText(frame, "Do you want to play again? (y/n)", (X, Y+100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                            
                            cv2.imshow("Frame", frame)
                            PlayAgain = cv2.waitKey(1) & 0xFF
                            if PlayAgain == ord('n'):
                                Play = False
                                break
                            elif PlayAgain == ord('y'):
                                break
                    else:
                        while True:
                            success, frame = video.read()
                            
                            X, Y = CenterTheText(frame, "No Hand Detected or the move is not recognized", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
                            cv2.putText(frame, "No Hand Detected or the move is not recognized", (X, Y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                            X, Y = CenterTheText(frame, "Do you want to play again? (y/n)", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
                            cv2.putText(frame, "Do you want to play again? (y/n)", (X, Y+100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                            cv2.imshow("Frame", frame)
                            PlayAgain = cv2.waitKey(1) & 0xFF
                            if PlayAgain == ord('n'):
                                Play = False
                                break
                            elif PlayAgain == ord('y'):
                                break
cv2.destroyAllWindows()
video.release()